In [25]:
from __future__ import print_function, division
import matplotlib.pyplot as plt
import pandas as pd

from neuralnilm.data.loadactivations import load_nilmtk_activations
from neuralnilm.data.syntheticaggregatesource import SyntheticAggregateSource
from neuralnilm.data.datapipeline import DataPipeline
from neuralnilm.data.processing import DivideBy
from neuralnilm.data.datathread import DataThread

In [26]:
NILMTK_FILENAME = '/data/mine/vadeec/merged/ukdale.h5'
TARGET_APPLIANCE = 'kettle'
SEQ_LENGTH = 256
SAMPLE_PERIOD = 6
STRIDE = 4

LOADER_CONFIG = {
    'nilmtk_activations': dict(
        appliances=['kettle', 'microwave', 'washing machine'],
        filename=NILMTK_FILENAME,
        sample_period=SAMPLE_PERIOD,
        windows={
            'train': {
                1: ("2014-01-01", "2014-02-01")
            },
            'unseen_activations_of_seen_appliances': {
                1: ("2014-02-02", "2014-02-08")                
            },
            'unseen_appliances': {
                2: ("2013-06-01", "2013-06-07")
            }
        }
    )
}

nilmtk_activations = load_nilmtk_activations(**LOADER_CONFIG['nilmtk_activations'])

INFO:neuralnilm.data.loadactivations:Loading NILMTK activations...
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 112 kettle activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 119 microwave activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading washing machine for UK-DALE_building_1...
INFO:neuralnilm.data.loadactivations:Loaded 23 washing machine activations from UK-DALE_building_1.
INFO:neuralnilm.data.loadactivations:Loading kettle for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 31 kettle activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading microwave for UK-DALE_building_2...
INFO:neuralnilm.data.loadactivations:Loaded 30 microwave activations from UK-DALE_building_2.
INFO:neuralnilm.data.loadactivations:Loading washing mac

In [27]:
nilmtk_activations.keys()

['train', 'unseen_activations_of_seen_appliances', 'unseen_appliances']

In [28]:
source = SyntheticAggregateSource(
    activations=nilmtk_activations,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    allow_incomplete_target=False
)

In [29]:
FOLD = 'train'
#FOLD = 'unseen_activations_of_seen_appliances'
#FOLD = 'unseen_appliances'
seq = source.get_sequence(enable_all_appliances=True)

In [30]:
#all(seq.all_appliances.sum(axis=1) == seq.input[:, 0])

In [31]:
#fig, axes = plt.subplots(2, sharex=True)
#seq.all_appliances.plot(ax=axes[0])
#axes[1].plot(seq.input)
#fig.suptitle(FOLD)
#plt.show()

In [32]:
sample = source.get_batch(num_seq_per_batch=1024)

In [33]:
pipeline = DataPipeline(
    [source],
    num_seq_per_batch=64,
    input_processing=[DivideBy(sample.before_processing.input.flatten().std())],
    target_processing=[DivideBy(sample.before_processing.target.flatten().std())]
)

# Disaggregation

nilmtk_disag_source = NILMTKDisagSource(
    filename=NILMTK_FILENAME,
    target_appliance=TARGET_APPLIANCE,
    seq_length=SEQ_LENGTH,
    buildings=[5],
    window_per_building={},
    stride=STRIDE,
    sample_period=SAMPLE_PERIOD
)

disag_pipeline = deepcopy(pipeline)
disag_pipeline.source = nilmtk_disag_source

disaggregator = Disaggregator(
    pipeline=disag_pipeline,
    output_path=PATH  # 
)

Disagregator ideas:

* make a copy of pipeline but swap source for a NILMTKDisagSource
* NILMTKDisagSource loads all data into memory (?) and iterates over chunks of it (get seq_length from pipeline.source.seq_length)
* 

In [34]:
report = pipeline.report()
report['activations'] = LOADER_CONFIG
report

{'DataPipeline': {'SyntheticAggregateSource': {'allow_incomplete_distractors': True,
   'allow_incomplete_target': False,
   'distractor_inclusion_prob': 1.0,
   'include_incomplete_target_in_output': True,
   'rng_seed': None,
   'seq_length': 256,
   'target_appliance': 'kettle',
   'target_inclusion_prob': 1.0,
   'uniform_prob_of_selecting_each_model': True},
  'input_processing': [{'DivideBy': {'divisor': 978.54016}}],
  'num_seq_per_batch': 64,
  'target_processing': [{'DivideBy': {'divisor': 593.93707}}]},
 'activations': {'nilmtk_activations': {'appliances': ['kettle',
    'microwave',
    'washing machine'],
   'filename': '/data/mine/vadeec/merged/ukdale.h5',
   'sample_period': 6,
   'windows': {'train': {1: ('2014-01-01', '2014-02-01')},
    'unseen_activations_of_seen_appliances': {1: ('2014-02-02', '2014-02-08')},
    'unseen_appliances': {2: ('2013-06-01', '2013-06-07')}}}}}

In [35]:
from lasagne.layers import InputLayer, RecurrentLayer, DenseLayer, ReshapeLayer

def get_net_0(input_shape, target_shape=None):
    NUM_UNITS = {
        'dense_layer_0': 100,
        'dense_layer_1':  50,
        'dense_layer_2': 100
    }

    if target_shape is None:
        target_shape = input_shape
    
    # Define layers
    input_layer = InputLayer(
        shape=input_shape
    )
    
    # Dense layers
    dense_layer_0 = DenseLayer(
        input_layer, 
        num_units=NUM_UNITS['dense_layer_0']
    )
    dense_layer_1 = DenseLayer(
        dense_layer_0, 
        num_units=NUM_UNITS['dense_layer_1']
    )
    dense_layer_2 = DenseLayer(
        dense_layer_1, 
        num_units=NUM_UNITS['dense_layer_2']
    )
    
    # Output
    final_dense_layer = DenseLayer(
        dense_layer_0,
        num_units=target_shape[1] * target_shape[2],
        nonlinearity=None
    )
    output_layer = ReshapeLayer(
        final_dense_layer,
        shape=target_shape
    )
    return output_layer

In [36]:
from neuralnilm.net import Net

batch = pipeline.get_batch()
output_layer = get_net_0(
    batch.after_processing.input.shape, 
    batch.after_processing.target.shape
)
net = Net(output_layer)

In [37]:
from pymongo import MongoClient
client = MongoClient()
client.drop_database('neuralnilm_experiments')
db = client.neuralnilm_experiments

In [38]:
from neuralnilm.trainer import Trainer
from neuralnilm.metrics import Metrics

trainer = Trainer(
    net=net,
    data_pipeline=pipeline,
    db=db,
    experiment_id=0,
    metrics=Metrics([4]),
    learning_rate=1E-2,
    repeat_callbacks=[
        ( 100, Trainer.validate)
    ]
)

INFO:neuralnilm.trainer:Iteration 0: Change learning rate to 1.0E-02


In [39]:
trainer.fit(400)

INFO:neuralnilm.trainer:Starting training for 400 iterations.
INFO:neuralnilm.trainer:Compiling train cost function...
INFO:neuralnilm.trainer:Done compiling cost function.
INFO:neuralnilm.net:Compiling deterministic output function...
INFO:neuralnilm.net:Done compiling deterministic output function.
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:958: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
INFO:neuralnilm.trainer:Stopped training. Completed 400 iterations.


In [40]:
net.train_iterations

400

In [41]:
import pymongo
list(db.train_scores.find(
        filter={'experiment_id': 0},
        sort=[('iteration', pymongo.ASCENDING)],
        projection=['iteration', 'loss']
    ).limit(50))

[{u'_id': ObjectId('55db6750a0777c0aa6d655f1'),
  u'iteration': 0,
  u'loss': 1.953479528427124},
 {u'_id': ObjectId('55db6750a0777c0aa6d655f8'),
  u'iteration': 1,
  u'loss': 1.753095269203186},
 {u'_id': ObjectId('55db6750a0777c0aa6d655f9'),
  u'iteration': 2,
  u'loss': 1.7143964767456055},
 {u'_id': ObjectId('55db6750a0777c0aa6d655fa'),
  u'iteration': 3,
  u'loss': 1.601704716682434},
 {u'_id': ObjectId('55db6750a0777c0aa6d655fb'),
  u'iteration': 4,
  u'loss': 1.5702732801437378},
 {u'_id': ObjectId('55db6750a0777c0aa6d655fc'),
  u'iteration': 5,
  u'loss': 1.341712474822998},
 {u'_id': ObjectId('55db6750a0777c0aa6d655fd'),
  u'iteration': 6,
  u'loss': 1.3802531957626343},
 {u'_id': ObjectId('55db6750a0777c0aa6d655fe'),
  u'iteration': 7,
  u'loss': 1.1945514678955078},
 {u'_id': ObjectId('55db6750a0777c0aa6d655ff'),
  u'iteration': 8,
  u'loss': 1.3421692848205566},
 {u'_id': ObjectId('55db6750a0777c0aa6d65600'),
  u'iteration': 9,
  u'loss': 1.2556569576263428},
 {u'_id': Obje

In [42]:
list(db.validation_scores.find({'fold': 'train', 'metric_type': 'classification'}).sort('iteration', pymongo.ASCENDING).limit(50))

[{u'_id': ObjectId('55db6750a0777c0aa6d655f2'),
  u'experiment_id': 0,
  u'fold': u'train',
  u'iteration': 0,
  u'metric_type': u'classification',
  u'num_states': 2,
  u'scores': {u'accuracy_score': 0.97998046875,
   u'f1_score': 0.0,
   u'precision_score': 0.0,
   u'recall_score': 0.0},
  u'source_name': u'SyntheticAggregateSource'},
 {u'_id': ObjectId('55db6754a0777c0aa6d6565c'),
  u'experiment_id': 0,
  u'fold': u'train',
  u'iteration': 100,
  u'metric_type': u'classification',
  u'num_states': 2,
  u'scores': {u'accuracy_score': 0.9886474609375,
   u'f1_score': 0.0,
   u'precision_score': 0.0,
   u'recall_score': 0.0},
  u'source_name': u'SyntheticAggregateSource'},
 {u'_id': ObjectId('55db6759a0777c0aa6d656c6'),
  u'experiment_id': 0,
  u'fold': u'train',
  u'iteration': 200,
  u'metric_type': u'classification',
  u'num_states': 2,
  u'scores': {u'accuracy_score': 0.98699951171875,
   u'f1_score': 0.0,
   u'precision_score': 0.0,
   u'recall_score': 0.0},
  u'source_name': u'Sy

In [43]:
agg_pipeline = [
    {"$group": {
            "_id": "$experiment_id",
            "max_acc": {"$max": "$SyntheticAggregateSource.unseen_appliances.2_state_classification.accuracy_score"},
            "max_iteration": {"$max": "$iteration"}
        }}
]
list(db.validation_scores.aggregate(agg_pipeline))

[{u'_id': 0, u'max_acc': None, u'max_iteration': 400}]

In [44]:
from monary import Monary

In [45]:
monary = Monary()

In [46]:
iterations, loss = monary.query(
    db='neuralnilm_experiments',
    coll='train_scores',
    query={'experiment_id': 0},
    fields=['iteration', 'loss'],
    types=['int32', 'float32']
)

In [47]:
import matplotlib.pyplot as plt
plt.plot(iterations, loss)
plt.show()

In [48]:
iterations, f1_score = monary.query(
    db='neuralnilm_experiments',
    coll='validation_scores',
    query={
        'experiment_id': 0, 
        'source_name': 'SyntheticAggregateSource',
        'fold': 'train',
        'metric_type': 'classification'
    },
    fields=['iteration', 'scores.f1_score'],
    types=['int64', 'float32']
)

In [49]:
f1_score

masked_array(data = [0.0 0.0 0.0 0.0 0.0],
             mask = [False False False False False],
       fill_value = 1e+20)

In [60]:
db.validation_scores.find_one(
    filter={
        'experiment_id': 0,
        'metric_type': 'classification'
    }
)['scores'].keys()

[u'f1_score', u'recall_score', u'accuracy_score', u'precision_score']

In [61]:
db.validation_scores.find_one(
    filter={
        'experiment_id': 1,
        'metric_type': 'classification'
    }
)